In [22]:
import pandas as pd
from eda_support_functions import *

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 1000)

In [24]:
RUNS_IDS = ['20240806_230924']

In [25]:
parameters_df = load_metadata('parameters', '/Users/iliasx/Documents/GitHub/box-office-prediction/metadata')
experiment_df = load_metadata('metrics', '/Users/iliasx/Documents/GitHub/box-office-prediction/metadata')

In [26]:
experiment_df

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
0,20240802_224457,2024-08-02 23:24:31,decision_tree_classifier,multi_class_classification,full,random_search,True,none,MinMaxScaler,0.011282,0.115336,None,0.498124,0.426014,0.498124,0.452741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240802_224457,2024-08-02 23:36:14,lightgbm_regressor,regression,full,random_search,True,complex,RobustScaler,0.019995,20.893485,None,NaN,NaN,NaN,NaN,1.372573e+16,1.364065,6.213266e+07,1.171569e+08,1.022013,1.044510,0.656524,0.162289,1.472104,1.679541,NaN
2,20240731_163830,2024-07-31 16:55:05,random_forest_regressor,regression,full,random_search,True,complex,StandardScaler,0.002653,993.781275,None,NaN,NaN,NaN,NaN,2.453859e+16,1.673388,6.192644e+07,1.566480e+08,1.108347,1.228433,0.529389,0.156367,1.819095,2.066391,NaN
3,20240719_214914,2024-07-19 22:31:22,decision_tree_classifier,binary_classification,small_productions,random_search,True,none,RobustScaler,0.104955,0.044046,None,0.571429,0.653061,0.561404,0.603774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.573385
4,20240724_210133,2024-07-24 21:03:13,decision_tree_regressor,regression,small_productions,random_search,False,none,StandardScaler,0.155938,0.713206,None,NaN,NaN,NaN,NaN,2.595991e+15,27.918844,1.471929e+07,5.095086e+07,2.341491,5.482579,0.035311,0.098639,29.208188,31.796131,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3153,20240729_230303,2024-07-30 00:12:45,dummy_classifier,binary_classification,medium_productions,random_search,True,none,RobustScaler,0.127511,0.401243,None,0.524116,0.476489,0.540925,0.506667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.525595
3154,20240806_230924,2024-08-06 23:21:52,xgboost_classifier,multi_class_classification,small_productions,random_search,False,complex,RobustScaler,0.018735,387.845553,None,0.615894,0.552778,0.615894,0.577467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3155,20240802_224457,2024-08-02 23:15:24,logistic_regression,multi_class_classification,small_productions,random_search,False,none,RobustScaler,0.019995,0.156260,None,0.587189,0.491697,0.587189,0.533433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3156,20240724_210133,2024-07-24 21:20:18,decision_tree_classifier,multi_class_classification,medium_productions,random_search,False,none,StandardScaler,0.155938,0.783357,None,0.335726,0.475243,0.335726,0.381156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
experiment_df.groupby('run_id').aggregate({'run_id': 'count',
                                          'dataset_name' : 'nunique',
                                          'model_type' : 'nunique',
                                          'timestamp' : 'max'}
                                          ).rename(columns={'run_id': 'count'}).sort_values('timestamp', ascending=False)

,count,dataset_name,model_type,timestamp
run_id,,,,
20240806_230924,67,4,11,2024-08-07 09:49:47
20240806_193816,96,4,6,2024-08-06 22:18:17
20240806_184222,4,4,1,2024-08-06 19:22:57
20240806_120055,2,2,1,2024-08-06 13:36:40
20240805_182129,2,2,1,2024-08-05 19:39:19
...,...,...,...,...
20240719_231427,12,2,6,2024-07-19 23:15:42
20240719_225631,2,2,1,2024-07-19 22:58:14
20240719_214914,272,4,11,2024-07-19 22:31:25


In [28]:
parameters_df = parameters_df[parameters_df['run_id'].isin(RUNS_IDS)]
experiment_df = experiment_df[experiment_df['run_id'].isin(RUNS_IDS)]

In [29]:
# parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'model__units', 'model__num_layers', 'model__dropout', 'epochs','variance_threshold','class_weight']].sort_values(by=['problem_type','dataset_name','has_outliers_removed'])

In [30]:
parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'scaler','variance_threshold','class_weight','C','min_child_samples','solver','num_leaves','n_estimators']].sort_values(by=['model_type','problem_type','dataset_name','has_outliers_removed'])

,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,class_weight,C,min_child_samples,solver,num_leaves,n_estimators
578,binary_classification,full,True,none,decision_tree_classifier,RobustScaler,0.074404,balanced,NaN,NaN,NaN,NaN,NaN
2654,binary_classification,full,True,complex,decision_tree_classifier,MinMaxScaler,0.083302,balanced,NaN,NaN,NaN,NaN,NaN
1392,binary_classification,large_productions,False,complex,decision_tree_classifier,MinMaxScaler,0.011639,balanced,NaN,NaN,NaN,NaN,NaN
2093,binary_classification,large_productions,False,none,decision_tree_classifier,RobustScaler,0.074404,balanced,NaN,NaN,NaN,NaN,NaN
272,binary_classification,medium_productions,True,complex,decision_tree_classifier,RobustScaler,0.079764,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,multi_class_classification,large_productions,True,complex,xgboost_classifier,RobustScaler,0.066180,None,NaN,NaN,NaN,NaN,829.0
3154,multi_class_classification,small_productions,False,complex,xgboost_classifier,RobustScaler,0.018735,NaN,NaN,NaN,NaN,NaN,737.0
2130,regression,large_productions,False,complex,xgboost_regressor,RobustScaler,0.027866,NaN,NaN,NaN,NaN,NaN,1807.0
2887,regression,small_productions,False,none,xgboost_regressor,MinMaxScaler,0.081791,NaN,NaN,NaN,NaN,NaN,1981.0


In [31]:
parameters_df[parameters_df['model_type'] == 'xgboost_regressor'][['timestamp','problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'scaler','variance_threshold', 'n_estimators', 'max_depth', 'learning_rate', 'num_leaves', 'min_child_samples']]

,timestamp,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,n_estimators,max_depth,learning_rate,num_leaves,min_child_samples
2130,2024-08-07 03:05:29,regression,large_productions,False,complex,xgboost_regressor,RobustScaler,0.027866,1807.0,5.0,0.063871,NaN,NaN
2682,2024-08-07 07:06:05,regression,small_productions,True,none,xgboost_regressor,StandardScaler,0.169734,834.0,27.0,0.060058,NaN,NaN
2887,2024-08-07 01:07:58,regression,small_productions,False,none,xgboost_regressor,MinMaxScaler,0.081791,1981.0,10.0,0.017175,NaN,NaN


In [32]:
parameters_df[parameters_df['model_type'] == 'lightgbm_regressor'][['timestamp','problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'scaler','variance_threshold', 'n_estimators', 'max_depth', 'learning_rate', 'num_leaves', 'min_child_samples']]

,timestamp,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,n_estimators,max_depth,learning_rate,num_leaves,min_child_samples
1169,2024-08-07 01:59:40,regression,small_productions,False,none,lightgbm_regressor,MinMaxScaler,0.126461,329.0,7.0,0.049432,84.0,20.0
1840,2024-08-07 03:50:28,regression,large_productions,False,complex,lightgbm_regressor,MinMaxScaler,0.006878,558.0,13.0,0.137653,23.0,20.0
2045,2024-08-07 07:34:10,regression,small_productions,True,none,lightgbm_regressor,MinMaxScaler,0.122371,1634.0,26.0,0.002413,234.0,20.0


In [33]:
parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'n_estimators', 'max_depth', 'learning_rate', 'variance_threshold','class_weight']].sort_values(by=['problem_type','dataset_name','has_outliers_removed'])

,problem_type,dataset_name,has_outliers_removed,feature_engineering,n_estimators,max_depth,learning_rate,variance_threshold,class_weight
138,binary_classification,full,True,complex,385.0,25.0,0.033587,0.005302,balanced
153,binary_classification,full,True,complex,NaN,NaN,NaN,0.045710,NaN
323,binary_classification,full,True,none,251.0,3.0,0.156027,0.178965,None
578,binary_classification,full,True,none,NaN,4.0,NaN,0.074404,balanced
626,binary_classification,full,True,complex,NaN,NaN,NaN,0.047197,balanced
...,...,...,...,...,...,...,...,...,...
1540,regression,small_productions,True,none,1837.0,4.0,NaN,0.034991,NaN
2045,regression,small_productions,True,none,1634.0,26.0,0.002413,0.122371,None
2192,regression,small_productions,True,none,NaN,NaN,NaN,0.159309,NaN
2682,regression,small_productions,True,none,834.0,27.0,0.060058,0.169734,NaN


In [34]:
problem_types = ['regression', 'binary_classification']
groupby_columns = ['dataset_name', 'has_outliers_removed', 'feature_engineering']
metrics = {
    'regression': {'metric': 'MAPE', 'operation': 'min'},
    'binary_classification': {'metric': 'F1 Score', 'operation': 'max'}
}
display_columns = {
    'regression': ['model_type', 'scaler', 'MAPE', 'Threshold Probability Accuracy', 'R2'],
    'binary_classification': ['model_type', 'scaler', 'F1 Score']
}

best_models = select_best_models(
    experiment_df,
    problem_types,
    groupby_columns,
    metrics,
    display_columns
)

# Access results
result_reg = best_models['regression']
result_class = best_models['binary_classification']

In [35]:
print(result_reg.to_string(index=False, justify='center', float_format='%.2f', decimal='.'))
print(result_class.to_string(index=False, justify='center', float_format='%.2f', decimal='.'))

problem_type    dataset_name    has_outliers_removed feature_engineering        model_type           scaler       MAPE   Threshold Probability Accuracy  R2 
 regression  large_productions         False               complex       decision_tree_regressor   MinMaxScaler    1.97               0.21              0.49
 regression  small_productions         False                  none            lightgbm_regressor   MinMaxScaler 1034.07               0.11              0.28
 regression  small_productions          True                  none       random_forest_regressor StandardScaler    1.82               0.09              0.13
     problem_type        dataset_name     has_outliers_removed feature_engineering        model_type            scaler      F1 Score
binary_classification               full          True               complex            lightgbm_classifier   RobustScaler    0.66  
binary_classification               full          True                  none            lightgbm_classifie

In [36]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='binary_classification', metric='F1 Score', metric_agg='max')


,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
138,20240806_230924,2024-08-07 07:04:15,lightgbm_classifier,binary_classification,full,random_search,True,complex,RobustScaler,0.005302,2905.855017,None,0.683962,0.630975,0.699153,0.663317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.685461
139,20240806_230924,2024-08-07 04:56:49,random_forest_classifier,binary_classification,large_productions,random_search,False,none,MinMaxScaler,0.091307,95.522064,None,0.683837,0.632411,0.653061,0.642570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.680304
153,20240806_230924,2024-08-07 06:03:49,dummy_classifier,binary_classification,full,random_search,True,complex,MinMaxScaler,0.045710,32.167884,None,0.512264,0.453416,0.463983,0.458639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.507502
169,20240806_230924,2024-08-07 02:00:06,dummy_classifier,binary_classification,large_productions,random_search,False,complex,MinMaxScaler,0.122371,24.920379,None,0.523979,0.456274,0.489796,0.472441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520055
272,20240806_230924,2024-08-07 07:37:37,decision_tree_classifier,binary_classification,medium_productions,random_search,True,complex,RobustScaler,0.079764,21.878734,None,0.584507,0.534483,0.492063,0.512397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575146
323,20240806_230924,2024-08-07 03:53:15,xgboost_classifier,binary_classification,full,random_search,True,none,MinMaxScaler,0.178965,39.227496,None,0.645283,0.604348,0.588983,0.596567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.639730
578,20240806_230924,2024-08-07 03:52:36,decision_tree_classifier,binary_classification,full,random_search,True,none,RobustScaler,0.074404,1.355146,None,0.629245,0.582463,0.591102,0.586751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.625483
626,20240806_230924,2024-08-07 06:05:09,logistic_regression,binary_classification,full,random_search,True,complex,MinMaxScaler,0.047197,79.547780,None,0.655660,0.607646,0.639831,0.623323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.654099
642,20240806_230924,2024-08-07 04:55:10,dummy_classifier,binary_classification,large_productions,random_search,False,none,RobustScaler,0.181653,0.833733,None,0.493783,0.429078,0.493878,0.459203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.493794
788,20240806_230924,2024-08-07 03:50:33,logistic_regression,binary_classification,full,random_search,True,none,StandardScaler,0.029617,3.723865,None,0.630189,0.576628,0.637712,0.605634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.630931


In [37]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='multi_class_classification', metric='F1 Score', metric_agg='max')

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
36,20240806_230924,2024-08-07 09:35:41,dummy_classifier,multi_class_classification,full,random_search,False,complex,StandardScaler,0.155938,40.723095,None,0.346632,0.341858,0.346632,0.344075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,20240806_230924,2024-08-07 08:09:25,xgboost_classifier,multi_class_classification,large_productions,random_search,True,complex,RobustScaler,0.066180,439.344465,None,0.530815,0.445009,0.530815,0.468214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,20240806_230924,2024-08-07 07:58:08,logistic_regression,multi_class_classification,large_productions,random_search,True,complex,MinMaxScaler,0.026549,85.620896,None,0.548708,0.466414,0.548708,0.480908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,20240806_230924,2024-08-06 23:11:45,logistic_regression,multi_class_classification,small_productions,random_search,False,complex,StandardScaler,0.013010,112.640552,None,0.582781,0.503448,0.582781,0.532265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,20240806_230924,2024-08-07 08:01:40,random_forest_classifier,multi_class_classification,large_productions,random_search,True,complex,StandardScaler,0.043164,211.814038,None,0.520875,0.449969,0.520875,0.472618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
748,20240806_230924,2024-08-07 09:34:59,lightgbm_classifier,multi_class_classification,large_productions,random_search,True,complex,StandardScaler,0.004863,5133.485016,None,0.558648,0.493411,0.558648,0.515212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
765,20240806_230924,2024-08-07 09:49:47,decision_tree_classifier,multi_class_classification,full,random_search,False,complex,MinMaxScaler,0.077083,43.842928,None,0.533028,0.416590,0.533028,0.467079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,20240806_230924,2024-08-07 01:04:19,lightgbm_classifier,multi_class_classification,small_productions,random_search,False,complex,MinMaxScaler,0.003432,6146.989130,None,0.629139,0.568863,0.629139,0.586257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
930,20240806_230924,2024-08-06 23:15:24,decision_tree_classifier,multi_class_classification,small_productions,random_search,False,complex,StandardScaler,0.042468,26.665997,None,0.562914,0.464881,0.562914,0.508063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1640,20240806_230924,2024-08-07 07:56:42,dummy_classifier,multi_class_classification,large_productions,random_search,True,complex,StandardScaler,0.074457,24.265600,None,0.347913,0.352211,0.347913,0.349945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='regression', metric='MAPE', metric_agg='min')

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
566,20240806_230924,2024-08-07 02:57:31,decision_tree_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.052179,30.404493,None,NaN,NaN,NaN,NaN,4.200325e+16,1.965304,1.124303e+08,2.049469e+08,1.098849,1.207470,0.492879,0.206039,2.176849,2.614239,NaN
1128,20240806_230924,2024-08-07 01:06:59,decision_tree_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.038855,1.417774,None,NaN,NaN,NaN,NaN,2.806414e+15,3716.256048,1.657437e+07,5.297560e+07,2.058231,4.236316,0.253129,0.104305,3951.790194,4283.604647,NaN
1169,20240806_230924,2024-08-07 01:59:40,lightgbm_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.126461,3102.059649,None,NaN,NaN,NaN,NaN,2.715088e+15,1034.074529,1.618450e+07,5.210650e+07,1.901545,3.615872,0.277433,0.105960,1088.117787,1194.206955,NaN
1540,20240806_230924,2024-08-07 07:05:39,random_forest_regressor,regression,small_productions,random_search,True,none,StandardScaler,0.034991,82.666126,None,NaN,NaN,NaN,NaN,5.250786e+14,1.819742,1.410932e+07,2.291459e+07,1.253378,1.570957,0.127607,0.091241,1.908518,2.059519,NaN
1737,20240806_230924,2024-08-07 02:37:56,dummy_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.028573,25.045260,None,NaN,NaN,NaN,NaN,9.061588e+16,24.565887,1.588650e+08,3.010247e+08,1.464467,2.144663,-0.094040,0.140320,26.290476,29.716493,NaN
1840,20240806_230924,2024-08-07 03:50:28,lightgbm_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.006878,2699.246889,None,NaN,NaN,NaN,NaN,4.284176e+16,4.328473,1.084042e+08,2.069825e+08,1.014492,1.029194,0.482755,0.223801,4.848679,5.829596,NaN
2045,20240806_230924,2024-08-07 07:34:10,lightgbm_regressor,regression,small_productions,random_search,True,none,MinMaxScaler,0.122371,1684.474341,None,NaN,NaN,NaN,NaN,4.955639e+14,1.945183,1.389599e+07,2.226126e+07,1.277452,1.631884,0.176644,0.105839,2.055481,2.205204,NaN
2130,20240806_230924,2024-08-07 03:05:29,xgboost_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.027866,477.792208,None,NaN,NaN,NaN,NaN,4.281868e+16,4.367213,1.059003e+08,2.069268e+08,1.029646,1.060172,0.483034,0.216696,4.854143,5.949208,NaN
2192,20240806_230924,2024-08-07 07:04:16,dummy_regressor,regression,small_productions,random_search,True,none,RobustScaler,0.159309,0.704337,None,NaN,NaN,NaN,NaN,7.406959e+14,3.826276,1.647679e+07,2.721573e+07,1.592469,2.535959,-0.230631,0.069343,3.954502,4.165196,NaN
2300,20240806_230924,2024-08-07 02:57:00,random_forest_regressor,regression,large_productions,random_search,False,complex,StandardScaler,0.037304,1143.638252,None,NaN,NaN,NaN,NaN,4.095737e+16,2.622140,1.044580e+08,2.023793e+08,1.018054,1.036433,0.505506,0.220249,3.029718,3.525503,NaN


In [39]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='regression', metric='Threshold Probability Accuracy', metric_agg='max')

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
566,20240806_230924,2024-08-07 02:57:31,decision_tree_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.052179,30.404493,None,NaN,NaN,NaN,NaN,4.200325e+16,1.965304,1.124303e+08,2.049469e+08,1.098849,1.207470,0.492879,0.206039,2.176849,2.614239,NaN
1128,20240806_230924,2024-08-07 01:06:59,decision_tree_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.038855,1.417774,None,NaN,NaN,NaN,NaN,2.806414e+15,3716.256048,1.657437e+07,5.297560e+07,2.058231,4.236316,0.253129,0.104305,3951.790194,4283.604647,NaN
1169,20240806_230924,2024-08-07 01:59:40,lightgbm_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.126461,3102.059649,None,NaN,NaN,NaN,NaN,2.715088e+15,1034.074529,1.618450e+07,5.210650e+07,1.901545,3.615872,0.277433,0.105960,1088.117787,1194.206955,NaN
1540,20240806_230924,2024-08-07 07:05:39,random_forest_regressor,regression,small_productions,random_search,True,none,StandardScaler,0.034991,82.666126,None,NaN,NaN,NaN,NaN,5.250786e+14,1.819742,1.410932e+07,2.291459e+07,1.253378,1.570957,0.127607,0.091241,1.908518,2.059519,NaN
1737,20240806_230924,2024-08-07 02:37:56,dummy_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.028573,25.045260,None,NaN,NaN,NaN,NaN,9.061588e+16,24.565887,1.588650e+08,3.010247e+08,1.464467,2.144663,-0.094040,0.140320,26.290476,29.716493,NaN
1840,20240806_230924,2024-08-07 03:50:28,lightgbm_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.006878,2699.246889,None,NaN,NaN,NaN,NaN,4.284176e+16,4.328473,1.084042e+08,2.069825e+08,1.014492,1.029194,0.482755,0.223801,4.848679,5.829596,NaN
2045,20240806_230924,2024-08-07 07:34:10,lightgbm_regressor,regression,small_productions,random_search,True,none,MinMaxScaler,0.122371,1684.474341,None,NaN,NaN,NaN,NaN,4.955639e+14,1.945183,1.389599e+07,2.226126e+07,1.277452,1.631884,0.176644,0.105839,2.055481,2.205204,NaN
2130,20240806_230924,2024-08-07 03:05:29,xgboost_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.027866,477.792208,None,NaN,NaN,NaN,NaN,4.281868e+16,4.367213,1.059003e+08,2.069268e+08,1.029646,1.060172,0.483034,0.216696,4.854143,5.949208,NaN
2192,20240806_230924,2024-08-07 07:04:16,dummy_regressor,regression,small_productions,random_search,True,none,RobustScaler,0.159309,0.704337,None,NaN,NaN,NaN,NaN,7.406959e+14,3.826276,1.647679e+07,2.721573e+07,1.592469,2.535959,-0.230631,0.069343,3.954502,4.165196,NaN
2300,20240806_230924,2024-08-07 02:57:00,random_forest_regressor,regression,large_productions,random_search,False,complex,StandardScaler,0.037304,1143.638252,None,NaN,NaN,NaN,NaN,4.095737e+16,2.622140,1.044580e+08,2.023793e+08,1.018054,1.036433,0.505506,0.220249,3.029718,3.525503,NaN


In [40]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='regression', metric='R2', metric_agg='max')

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
566,20240806_230924,2024-08-07 02:57:31,decision_tree_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.052179,30.404493,None,NaN,NaN,NaN,NaN,4.200325e+16,1.965304,1.124303e+08,2.049469e+08,1.098849,1.207470,0.492879,0.206039,2.176849,2.614239,NaN
1128,20240806_230924,2024-08-07 01:06:59,decision_tree_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.038855,1.417774,None,NaN,NaN,NaN,NaN,2.806414e+15,3716.256048,1.657437e+07,5.297560e+07,2.058231,4.236316,0.253129,0.104305,3951.790194,4283.604647,NaN
1169,20240806_230924,2024-08-07 01:59:40,lightgbm_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.126461,3102.059649,None,NaN,NaN,NaN,NaN,2.715088e+15,1034.074529,1.618450e+07,5.210650e+07,1.901545,3.615872,0.277433,0.105960,1088.117787,1194.206955,NaN
1540,20240806_230924,2024-08-07 07:05:39,random_forest_regressor,regression,small_productions,random_search,True,none,StandardScaler,0.034991,82.666126,None,NaN,NaN,NaN,NaN,5.250786e+14,1.819742,1.410932e+07,2.291459e+07,1.253378,1.570957,0.127607,0.091241,1.908518,2.059519,NaN
1737,20240806_230924,2024-08-07 02:37:56,dummy_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.028573,25.045260,None,NaN,NaN,NaN,NaN,9.061588e+16,24.565887,1.588650e+08,3.010247e+08,1.464467,2.144663,-0.094040,0.140320,26.290476,29.716493,NaN
1840,20240806_230924,2024-08-07 03:50:28,lightgbm_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.006878,2699.246889,None,NaN,NaN,NaN,NaN,4.284176e+16,4.328473,1.084042e+08,2.069825e+08,1.014492,1.029194,0.482755,0.223801,4.848679,5.829596,NaN
2045,20240806_230924,2024-08-07 07:34:10,lightgbm_regressor,regression,small_productions,random_search,True,none,MinMaxScaler,0.122371,1684.474341,None,NaN,NaN,NaN,NaN,4.955639e+14,1.945183,1.389599e+07,2.226126e+07,1.277452,1.631884,0.176644,0.105839,2.055481,2.205204,NaN
2130,20240806_230924,2024-08-07 03:05:29,xgboost_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.027866,477.792208,None,NaN,NaN,NaN,NaN,4.281868e+16,4.367213,1.059003e+08,2.069268e+08,1.029646,1.060172,0.483034,0.216696,4.854143,5.949208,NaN
2192,20240806_230924,2024-08-07 07:04:16,dummy_regressor,regression,small_productions,random_search,True,none,RobustScaler,0.159309,0.704337,None,NaN,NaN,NaN,NaN,7.406959e+14,3.826276,1.647679e+07,2.721573e+07,1.592469,2.535959,-0.230631,0.069343,3.954502,4.165196,NaN
2300,20240806_230924,2024-08-07 02:57:00,random_forest_regressor,regression,large_productions,random_search,False,complex,StandardScaler,0.037304,1143.638252,None,NaN,NaN,NaN,NaN,4.095737e+16,2.622140,1.044580e+08,2.023793e+08,1.018054,1.036433,0.505506,0.220249,3.029718,3.525503,NaN


In [41]:
plot_one_metric_of_different_models_per_dataset_with_plotly(experiment_df, problem_type='regression', metric='MAPE', metric_agg='min', benchmark_model='dummy_regressor')

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
566,20240806_230924,2024-08-07 02:57:31,decision_tree_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.052179,30.404493,None,NaN,NaN,NaN,NaN,4.200325e+16,1.965304,1.124303e+08,2.049469e+08,1.098849,1.207470,0.492879,0.206039,2.176849,2.614239,NaN
1128,20240806_230924,2024-08-07 01:06:59,decision_tree_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.038855,1.417774,None,NaN,NaN,NaN,NaN,2.806414e+15,3716.256048,1.657437e+07,5.297560e+07,2.058231,4.236316,0.253129,0.104305,3951.790194,4283.604647,NaN
1169,20240806_230924,2024-08-07 01:59:40,lightgbm_regressor,regression,small_productions,random_search,False,none,MinMaxScaler,0.126461,3102.059649,None,NaN,NaN,NaN,NaN,2.715088e+15,1034.074529,1.618450e+07,5.210650e+07,1.901545,3.615872,0.277433,0.105960,1088.117787,1194.206955,NaN
1540,20240806_230924,2024-08-07 07:05:39,random_forest_regressor,regression,small_productions,random_search,True,none,StandardScaler,0.034991,82.666126,None,NaN,NaN,NaN,NaN,5.250786e+14,1.819742,1.410932e+07,2.291459e+07,1.253378,1.570957,0.127607,0.091241,1.908518,2.059519,NaN
1737,20240806_230924,2024-08-07 02:37:56,dummy_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.028573,25.045260,None,NaN,NaN,NaN,NaN,9.061588e+16,24.565887,1.588650e+08,3.010247e+08,1.464467,2.144663,-0.094040,0.140320,26.290476,29.716493,NaN
1840,20240806_230924,2024-08-07 03:50:28,lightgbm_regressor,regression,large_productions,random_search,False,complex,MinMaxScaler,0.006878,2699.246889,None,NaN,NaN,NaN,NaN,4.284176e+16,4.328473,1.084042e+08,2.069825e+08,1.014492,1.029194,0.482755,0.223801,4.848679,5.829596,NaN
2045,20240806_230924,2024-08-07 07:34:10,lightgbm_regressor,regression,small_productions,random_search,True,none,MinMaxScaler,0.122371,1684.474341,None,NaN,NaN,NaN,NaN,4.955639e+14,1.945183,1.389599e+07,2.226126e+07,1.277452,1.631884,0.176644,0.105839,2.055481,2.205204,NaN
2130,20240806_230924,2024-08-07 03:05:29,xgboost_regressor,regression,large_productions,random_search,False,complex,RobustScaler,0.027866,477.792208,None,NaN,NaN,NaN,NaN,4.281868e+16,4.367213,1.059003e+08,2.069268e+08,1.029646,1.060172,0.483034,0.216696,4.854143,5.949208,NaN
2192,20240806_230924,2024-08-07 07:04:16,dummy_regressor,regression,small_productions,random_search,True,none,RobustScaler,0.159309,0.704337,None,NaN,NaN,NaN,NaN,7.406959e+14,3.826276,1.647679e+07,2.721573e+07,1.592469,2.535959,-0.230631,0.069343,3.954502,4.165196,NaN
2300,20240806_230924,2024-08-07 02:57:00,random_forest_regressor,regression,large_productions,random_search,False,complex,StandardScaler,0.037304,1143.638252,None,NaN,NaN,NaN,NaN,4.095737e+16,2.622140,1.044580e+08,2.023793e+08,1.018054,1.036433,0.505506,0.220249,3.029718,3.525503,NaN


In [42]:
plot_one_metric_of_different_models_per_dataset_with_plotly(experiment_df, problem_type='binary_classification', metric='F1 Score', metric_agg='max', benchmark_model='dummy_classifier')

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
138,20240806_230924,2024-08-07 07:04:15,lightgbm_classifier,binary_classification,full,random_search,True,complex,RobustScaler,0.005302,2905.855017,None,0.683962,0.630975,0.699153,0.663317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.685461
139,20240806_230924,2024-08-07 04:56:49,random_forest_classifier,binary_classification,large_productions,random_search,False,none,MinMaxScaler,0.091307,95.522064,None,0.683837,0.632411,0.653061,0.642570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.680304
153,20240806_230924,2024-08-07 06:03:49,dummy_classifier,binary_classification,full,random_search,True,complex,MinMaxScaler,0.045710,32.167884,None,0.512264,0.453416,0.463983,0.458639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.507502
169,20240806_230924,2024-08-07 02:00:06,dummy_classifier,binary_classification,large_productions,random_search,False,complex,MinMaxScaler,0.122371,24.920379,None,0.523979,0.456274,0.489796,0.472441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520055
272,20240806_230924,2024-08-07 07:37:37,decision_tree_classifier,binary_classification,medium_productions,random_search,True,complex,RobustScaler,0.079764,21.878734,None,0.584507,0.534483,0.492063,0.512397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575146
323,20240806_230924,2024-08-07 03:53:15,xgboost_classifier,binary_classification,full,random_search,True,none,MinMaxScaler,0.178965,39.227496,None,0.645283,0.604348,0.588983,0.596567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.639730
578,20240806_230924,2024-08-07 03:52:36,decision_tree_classifier,binary_classification,full,random_search,True,none,RobustScaler,0.074404,1.355146,None,0.629245,0.582463,0.591102,0.586751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.625483
626,20240806_230924,2024-08-07 06:05:09,logistic_regression,binary_classification,full,random_search,True,complex,MinMaxScaler,0.047197,79.547780,None,0.655660,0.607646,0.639831,0.623323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.654099
642,20240806_230924,2024-08-07 04:55:10,dummy_classifier,binary_classification,large_productions,random_search,False,none,RobustScaler,0.181653,0.833733,None,0.493783,0.429078,0.493878,0.459203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.493794
788,20240806_230924,2024-08-07 03:50:33,logistic_regression,binary_classification,full,random_search,True,none,StandardScaler,0.029617,3.723865,None,0.630189,0.576628,0.637712,0.605634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.630931
